##### Imports:

In [1]:
import glob, re, pickle, os, pandas, rpy2, time, math, yaml, numpy, sklearn
import rpy2.robjects as ro
from odict import odict
from rpy2.robjects.packages import importr
calR = importr('CalibrationCurves')
grdevices = importr('grDevices')


### 1. Choose the Experiment Set

Choose the top directory of the experiments from which you want to plot the results.
Also, define an output directory to which to write the tables and plots.

In [2]:
# Define in and output directories

top_prefix="/hpc/shared/julius_te/tleeuwenberg/collin-out/"
top_level_experiments_output_dir="/sims-out/"
top_level_out_dir="../../../tables_and_plots/"
#top_level_out_dir = "/sims-13-11-2020/"
models_to_include = ['LR','Lasso','Ridge','ElasticNet','PCLR','LAELR','Dropout','LRnn']
metrics_to_include = ['C (ROC)','Slope','R2','ExpJacc','ExpSE','Intercept','MSE','logMSE']
coeff_metrics_to_include = ['sum_neg_oar','sum_pos_oar','perc_oar_neg','perc_oar_pos']
CI=0.95
# Metrics could be: 'AUROC','R2', 'Brier', 'ScaledBrier', 'Cintercept', 'Cslope', 'ECI'
CI_dev = (1.0 - CI) / 2

In [3]:
oar_coeff_substrings = ["Submandibular_", "Parotid_", "PCM_","Supraglottic_", "OralCavity_", "GlotticArea_"]

### 2. Extracting Results

The code below extracts the relevant information from the underlying subdirectories and output files (may take a few minutes).

In [4]:
# relative paths to locate relevant files
search_path = top_prefix+top_level_experiments_output_dir+"/"
bootrap_path_extension = "/**/bs*/*"
model_preds_path = '/pred_probs/model_preds.pickle'
true_labels_path= '/pred_probs/true_labels.pickle'
gt_model_path = '/sim/sim_gt_model.p'
test_probs = '/sim/test_probs.p'
train_data = '/sim/train_data.p'
rval_path ='/rvals/rvals_per_model.p'

model_coef_path = '/models.csv'
trained_models_path='/trained_models.p'
yaml_config_path = '/../../../config.yml'

results = odict()
       
        # --- Results Structure ---
        # results > $exp > preds > $model_name > $bs > $pred_probs
        # results  > $exp > labs > $bs > $true_labs
        # results > $exp > models > $model_name > $coeffs > $bs > $coef_value
        # results > $exp > bs_ixs > $bootstrap_indices
        # results > $exp > yaml > $yaml_dict
print('total:',len(list(glob.glob(search_path + bootrap_path_extension, recursive=True))))
i=0        
for bs_path in glob.glob(search_path + bootrap_path_extension, recursive=True):
    i+=1
    # extract experiment name and bootstrap number from the path
    exp_name = bs_path.split('/')[-1]
    bs = int(re.match(r'.*/bs-(\d+)/.*',bs_path).groups()[0])
    print(str(i), end = ' ,')
    #print('>',exp_name, ' bootrap:', bs)
    
    # setup exp dict
    if not exp_name in results:
        results[exp_name] = {'bs_path':{}, 'preds':{},'labs':{}, 'models':{},'gt_models':{},'gt_test_probs':{}, 'bs_ixs':[],'rvals':{},'pymodels':{},'train_labs':{}}

    results[exp_name]['bs_path'][bs] = bs_path
    
    # get the bs index
    if not bs in results[exp_name]['bs_ixs']:
        results[exp_name]['bs_ixs'].append(bs)
    
    # get the yaml config
    if os.path.exists(bs_path + yaml_config_path):
        with open(bs_path + yaml_config_path) as config_file:
            results[exp_name]['yaml'] = yaml.full_load(config_file)
    
    # Get the model PREDICTIONS (per bs per model) - if available in model_preds_path
    if os.path.exists(bs_path + model_preds_path):
        with open(bs_path + model_preds_path, 'rb') as f:
            for model_name, model_predictions in pickle.load(f).items():
                if not model_name in results[exp_name]['preds']:
                    results[exp_name]['preds'][model_name] = {}
                results[exp_name]['preds'][model_name][bs] = [float(p) for p in model_predictions]
    
    # Get the true LABELS (per bs) - if available in true_labels_path
    if os.path.exists(bs_path + true_labels_path):
        with open(bs_path + true_labels_path, 'rb') as f:
            if not bs in results[exp_name]['labs']:
                results[exp_name]['labs'][bs] = pickle.load(f)
            else:
                # if there is already a set of true labels for this bs and exp, check if they are the same
                if not pickle.load(f) == results[exp_name]['labs'][bs]:
                    print('WARNING: labels are not consistent between models for exp', exp_name, ' bs', bs)
    
    # Get the already validated predictions
    if os.path.exists(bs_path + rval_path):
        with open(bs_path + rval_path, 'rb') as f:
            for model_name, model_rval in pickle.load(f).items():
                    if not model_name in results[exp_name]['rvals']:
                        results[exp_name]['rvals'][model_name] = {}
                    results[exp_name]['rvals'][model_name][bs] = model_rval

    # Get the model coefficients (per bs per model)
    if os.path.exists(bs_path + model_coef_path):
        models_coeff_df = pandas.read_csv(bs_path + model_coef_path, sep='\t',index_col=0, header=0).filter(regex=r".*_mean")
        
        # get coef names (columns with _mean in it)
        coef_names = models_coeff_df.columns

        # get the coefficient values
        for row in models_coeff_df.iterrows():
            model_name = row[0].split('-')[-1]
            if not model_name in results[exp_name]['models']:
                results[exp_name]['models'][model_name] = {coef_name:{} for coef_name in coef_names}
            for coef_name in results[exp_name]['models'][model_name]:
                results[exp_name]['models'][model_name][coef_name][bs] = row[1][coef_name]

                
    # SIMULATION 
    # Get GT model 
    if os.path.exists(bs_path + gt_model_path):
        with open(bs_path + gt_model_path, 'rb') as f:
            results[exp_name]['gt_models'][bs] = pickle.load(f)

    # Get Simulated test data 
    if os.path.exists(bs_path + test_probs):
        with open(bs_path + test_probs, 'rb') as f:
            results[exp_name]['gt_test_probs'][bs] = pickle.load(f)

    # Train data
    if os.path.exists(bs_path + train_data):
        with open(bs_path + train_data, 'rb') as f:
            train_df = pickle.load(f).filter(regex=("^y_*"))
            #print(train_df.keys())
            results[exp_name]['train_labs'][bs] = list(train_df.values)                       
            
# Plotting overview
print('\n\nExperiments:',len(results))
for exp in sorted(results):
    print(exp, '(', len(results[exp]['bs_ixs']),'bs )')

    
    


total: 800
1 ,2 ,3 ,4 ,5 ,6 ,7 ,8 ,9 ,10 ,11 ,12 ,13 ,14 ,15 ,16 ,17 ,18 ,19 ,20 ,21 ,22 ,23 ,24 ,25 ,26 ,27 ,28 ,29 ,30 ,31 ,32 ,33 ,34 ,35 ,36 ,37 ,38 ,39 ,40 ,41 ,42 ,43 ,44 ,45 ,46 ,47 ,48 ,49 ,50 ,51 ,52 ,53 ,54 ,55 ,56 ,57 ,58 ,59 ,60 ,61 ,62 ,63 ,64 ,65 ,66 ,67 ,68 ,69 ,70 ,71 ,72 ,73 ,74 ,75 ,76 ,77 ,78 ,79 ,80 ,81 ,82 ,83 ,84 ,85 ,86 ,87 ,88 ,89 ,90 ,91 ,92 ,93 ,94 ,95 ,96 ,97 ,98 ,99 ,100 ,101 ,102 ,103 ,104 ,105 ,106 ,107 ,108 ,109 ,110 ,111 ,112 ,113 ,114 ,115 ,116 ,117 ,118 ,119 ,120 ,121 ,122 ,123 ,124 ,125 ,126 ,127 ,128 ,129 ,130 ,131 ,132 ,133 ,134 ,135 ,136 ,137 ,138 ,139 ,140 ,141 ,142 ,143 ,144 ,145 ,146 ,147 ,148 ,149 ,150 ,151 ,152 ,153 ,154 ,155 ,156 ,157 ,158 ,159 ,160 ,161 ,162 ,163 ,164 ,165 ,166 ,167 ,168 ,169 ,170 ,171 ,172 ,173 ,174 ,175 ,176 ,177 ,178 ,179 ,180 ,181 ,182 ,183 ,184 ,185 ,186 ,187 ,188 ,189 ,190 ,191 ,192 ,193 ,194 ,195 ,196 ,197 ,198 ,199 ,200 ,201 ,202 ,203 ,204 ,205 ,206 ,207 ,208 ,209 ,210 ,211 ,212 ,213 ,214 ,215 ,216 ,217 ,218 ,219 ,22

### Getting info on simulation settings (GT AUROC, no. predictors, no. events, EPV)

In [5]:


print('\t'.join(['Exp','AUROC','Preds','Events','EPV']))
sorted_exps = sorted(list(results.keys()))
for exp_name in sorted_exps:
    
    gt_aurocs = []
    no_events = []
    no_preds = len(results[exp_name]['gt_models'][0].best_estimator_.coef_[0])
    epv = []
    for bs in results[exp_name]['gt_test_probs']:
        
        if bs in results[exp_name]['labs']:
            auroc = sklearn.metrics.roc_auc_score(results[exp_name]['labs'][bs],results[exp_name]['gt_test_probs'][bs])
            gt_aurocs.append(auroc)
            no_events.append(sum(results[exp_name]['train_labs'][bs]))
            
            epv.append(sum(results[exp_name]['train_labs'][bs]) / no_preds)
            #print(results[exp_name]['train_labs'][bs])
    print('\t'.join([exp_name.replace('CompareMethods:sim-',''),str(round(numpy.mean(gt_aurocs),2)),str(no_preds),str(round(numpy.mean(no_events),2)),str(round(numpy.mean(epv),2))]))
        

Exp	AUROC	Preds	Events	EPV
A	0.79	7	159.54	22.79
A'	0.8	7	160.14	22.88
B	0.79	19	159.64	8.4
B'	0.8	19	160.03	8.42
C	0.87	13	83.33	6.41
C'	0.87	13	83.0	6.38
D	0.85	43	83.54	1.94
D'	0.85	43	83.59	1.94


In [6]:
#metrics_to_include = ['C (ROC)','Slope','R2','ExpJacc','ExpSE','Intercept','logMSE'] # remove MSE



### 3.1. Calculating predictive performance evaluations
May take a few minutes....

In [6]:


# Based on CalibrationCurve library in R
from sklearn.metrics import jaccard_score
import random

rval = ro.r['val.prob.ci.2']
#R metrics: 'Dxy', 'C (ROC)', 'R2', 'D', 'D:Chi-sq', 'D:p', 'U', 'U:Chi-sq', 'U:p', 'Q', 'Brier', 'Intercept', 'Slope', 'Emax', 'Brier scaled'


sign = lambda coeffs: [1.0 if c >=0.01 else -1.0 if c <= -0.01 else 0.0 for c in coeffs]

for exp_name in results:
    print(exp_name)
    #break
    
    # Setup output dir
    plot_path = top_level_out_dir + '/calib_plots/' 
    if not os.path.exists(os.path.dirname(plot_path)):
        os.makedirs(os.path.dirname(plot_path))
    
    
    # 1. Calculate the scores for all bootstraps
    scores = {model:{metric:{} for metric in metrics_to_include} for model in models_to_include}
    results[exp_name]['inclusion_vectors'] = {model:{} for model in results[exp_name]['models']}
    results[exp_name]['true_inclusion_vectors'] = {}
        
    for bs in results[exp_name]['bs_ixs']:
        print(bs,end=', ')    
        
        if 'sim' in exp_name:
            gt_model_coeffs = numpy.array(results[exp_name]['gt_models'][bs].best_estimator_.coef_).flatten()
            true_inclusion_vec = sign(gt_model_coeffs)        
            results[exp_name]['true_inclusion_vectors'][bs] = true_inclusion_vec
        
        for model in scores:
            # for each model inclusion vecs are constructed and saved
            coeff_names = [c for c in results[exp_name]['models'][model].keys() if not c=='Intercept_mean']
            inclusion_vec = sign([results[exp_name]['models'][model][coeff_name][bs] for coeff_name in coeff_names])
            results[exp_name]['inclusion_vectors'][model][bs] = inclusion_vec
            
            # for each model performance evaluation metrics are calculated and saved 
            if bs in results[exp_name]['preds'][model]:
                # ! Metrics are calculated here (currently using the val.prob.ci.2' R function)
                val = results[exp_name]['rvals'][model][bs]
                rval_dict = dict(zip(results[exp_name]['rvals'][model][bs].names, list(results[exp_name]['rvals'][model][bs])))
                for metric, v in rval_dict.items():
                    if not metric in scores[model]:
                        scores[model][metric] = {}
                    scores[model][metric][bs] = v
                    
        # for each bs, the trained models are loaded and MSE is calculated per method
        trained_models_path = results[exp_name]['bs_path'][bs] + '/trained_models.p'
        if os.path.exists(trained_models_path) and bs in results[exp_name]['gt_models']:
            with open(trained_models_path,'rb') as f:
                for model_name, trained_models in pickle.load(f).items():
                    mses = []
                    for trained_model in trained_models:

                        model_coeffs = numpy.array(trained_model.best_estimator_.coef_).flatten()
                        true_coeffs = numpy.array(results[exp_name]['gt_models'][bs].best_estimator_.coef_).flatten()

                        mse = numpy.mean((true_coeffs-model_coeffs)**2)

                        mses.append(mse)
                    scores[model_name]['MSE'][int(bs)] = numpy.mean(mses)
                    scores[model_name]['logMSE'][int(bs)] = numpy.log(numpy.mean(mses))
        
    
    # Approximating the expected overlap in selected coefficients (and has the same sign) when repeating the experiment, for each method.
    for model in results[exp_name]['inclusion_vectors']:
        scores[model]['ExpJacc'] = {}
        scores[model]['ExpSE'] = {}
        
        incl_vec_list = list(results[exp_name]['inclusion_vectors'][model].values())
        
        coeffs_per_bs = {bs: numpy.array([results[exp_name]['models'][model][coeff_name][bs] for coeff_name in coeff_names]) for bs in results[exp_name]['bs_ixs']}
        mean_coeff = numpy.mean(list(coeffs_per_bs.values()),axis=0)
        diffs = [sklearn.metrics.mean_squared_error(coeff,mean_coeff) for coeff in coeffs_per_bs.values()]
        
        for bs in results[exp_name]['bs_ixs']:
            scores[model]['ExpSE'][bs] = numpy.mean(random.choices(diffs, k=len(results[exp_name]['bs_ixs'])))
        
        for bs in results[exp_name]['bs_ixs']: #100 x
            sampled_incl_vec_list = random.choices(incl_vec_list, k=len(results[exp_name]['bs_ixs'])) 
            sims = []
            ses = []
            checked = set()
            # num_bootstraps x num_bootstraps / 2  
            for i,vec1 in enumerate(sampled_incl_vec_list):
                for j,vec2 in enumerate(sampled_incl_vec_list):
                    
                    if not (i==j or (i,j) in checked):
                            
                            # Mean jacc (based only on included coefficients only)
                            intersection_size = sum([1.0 if vec1[i]==vec2[i] and vec1[i]!=0.0 else 0.0 for i in range(len(vec1))])
                            union_size = sum([1 if vec1[i]!= 0 or vec2[i]!=0 else 0 for i in range(len(vec1))])

                            similarity = intersection_size / union_size
                            sims.append(similarity)

                            checked.add((i,j))
                            checked.add((j,i))
                        
            scores[model]['ExpJacc'][bs] = numpy.mean(sims)
    

        
            
    print('calculating CI')
    # 2. Determine the reported scores, and confidence intervals
    main_scores, CI_lower, CI_upper = {model:{} for model in models_to_include}, {model:{} for model in models_to_include}, {model:{} for model in models_to_include}
    for model in scores:
        for metric in metrics_to_include:
            sorted_scores = sorted(scores[model][metric].values())
            main_scores[model][metric] =  numpy.mean(list(scores[model][metric].values()))

            CI_lower_ix = int(CI_dev*len(scores[model][metric]))
            CI_upper_ix = int(math.ceil((1.0- CI_dev) * (len(scores[model][metric])-1)))
            CI_lower[model][metric]=sorted_scores[CI_lower_ix]
            CI_upper[model][metric]=sorted_scores[CI_upper_ix]
    
    # save results to large  results dictionary
    results[exp_name]['performance_scores'] = {'main':main_scores, 'ci_lower': CI_lower, 'ci_upper':CI_upper}
    results[exp_name]['performance_scores_all'] = scores

print('done')

CompareMethods:sim-A'
7, 65, 68, 99, 60, 66, 31, 72, 84, 13, 79, 53, 41, 9, 61, 49, 86, 25, 16, 88, 94, 80, 40, 52, 51, 24, 54, 3, 26, 89, 11, 71, 74, 23, 8, 46, 81, 95, 37, 82, 93, 32, 36, 58, 47, 73, 78, 2, 97, 44, 42, 22, 98, 64, 75, 76, 39, 28, 92, 12, 6, 63, 15, 29, 87, 67, 20, 34, 55, 21, 35, 62, 30, 1, 83, 4, 59, 10, 57, 90, 50, 85, 27, 5, 43, 91, 56, 17, 38, 33, 45, 70, 96, 18, 77, 69, 48, 14, 0, 19, calculating CI
CompareMethods:sim-D
7, 65, 68, 99, 60, 66, 31, 72, 84, 13, 79, 53, 41, 9, 61, 49, 86, 25, 16, 88, 94, 80, 40, 52, 51, 24, 54, 3, 26, 89, 11, 71, 74, 23, 8, 46, 81, 95, 37, 82, 93, 32, 36, 58, 47, 73, 78, 2, 97, 44, 42, 22, 98, 64, 75, 76, 39, 28, 92, 12, 6, 63, 15, 29, 87, 67, 20, 34, 55, 21, 35, 62, 30, 1, 83, 4, 59, 10, 57, 90, 50, 85, 27, 5, 43, 91, 56, 17, 38, 33, 45, 70, 96, 18, 77, 69, 48, 14, 0, 19, calculating CI
CompareMethods:sim-C
7, 65, 68, 99, 60, 66, 31, 72, 84, 13, 79, 53, 41, 9, 61, 49, 86, 25, 16, 88, 94, 80, 40, 52, 51, 24, 54, 3, 26, 89, 11, 71, 7

#### Calibration plots (fan plots)

In [ ]:
import sklearn.calibration
import statsmodels.api as sm
import matplotlib
import matplotlib.pyplot as plt
font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 18}
matplotlib.rc('font', **font)



calib_out_path = './calib_plots/'
lowess = sm.nonparametric.lowess
n_bins = 10
loess_frac=0.33
loess_it=3
strategy='quantile' # or quantile
for exp_name in results: 
    print(exp_name)
    if not os.path.exists(calib_out_path + '/' + exp_name + '/' ):
        os.makedirs(calib_out_path + '/' + exp_name + '/')
    for model in results[exp_name]['preds']:
        all_preds, all_labs = [],[]
        print(model)
        plot_name = exp_name + ':'+model+'.pdf'
        # start plot
        plt.axis([0, 1, 0, 1])
        slope = 'TBA'
        cs = round(results[exp_name]['performance_scores']['main'][model]['Slope'],2)
        cs_lower = results[exp_name]['performance_scores']['ci_lower'][model]['Slope']
        print(cs_lower)
        auroc = round(results[exp_name]['performance_scores']['main'][model]['C (ROC)'],2)
        r2 = round(results[exp_name]['performance_scores']['main'][model]['R2'],2)
        
        plt.text(0.35, .15, 'AUC=%0.2f (%0.2f, %0.2f)' % (results[exp_name]['performance_scores']['main'][model]['C (ROC)'], results[exp_name]['performance_scores']['ci_lower'][model]['C (ROC)'],results[exp_name]['performance_scores']['ci_upper'][model]['C (ROC)']))
        plt.text(0.35, 0.05, 'R²=%0.2f (%0.2f, %0.2f)' % (results[exp_name]['performance_scores']['main'][model]['R2'], results[exp_name]['performance_scores']['ci_lower'][model]['R2'],results[exp_name]['performance_scores']['ci_upper'][model]['R2']))

        plt.text(0.03, .9, 'Intercept=%0.2f (%0.2f, %0.2f)' % (results[exp_name]['performance_scores']['main'][model]['Intercept'], results[exp_name]['performance_scores']['ci_lower'][model]['Intercept'],results[exp_name]['performance_scores']['ci_upper'][model]['Intercept']))        
        plt.text(0.03, .8, 'Slope=%0.2f (%0.2f, %0.2f)' % (results[exp_name]['performance_scores']['main'][model]['Slope'], results[exp_name]['performance_scores']['ci_lower'][model]['Slope'],results[exp_name]['performance_scores']['ci_upper'][model]['Slope']))
        
        for bs in results[exp_name]['preds'][model]:
            prob_true, prob_pred = sklearn.calibration.calibration_curve(results[exp_name]['labs'][bs], results[exp_name]['preds'][model][bs], n_bins=n_bins, strategy=strategy)
            
            z = lowess(prob_true,prob_pred, frac=loess_frac,it=loess_it)

            plt.plot(z[:,0], z[:,1], alpha=0.1, color='k')
            all_preds += results[exp_name]['preds'][model][bs]
            all_labs += results[exp_name]['labs'][bs]

        full_prob_true, full_prob_pred = sklearn.calibration.calibration_curve(all_labs, all_preds, n_bins=2*n_bins, strategy=strategy)
        z = lowess(full_prob_true,full_prob_pred, frac=loess_frac,it=loess_it)

        plt.plot(z[:,0], z[:,1], alpha=1.0, color='b')
        plt.plot(numpy.arange(0,1+1.0/n_bins,1.0/n_bins),numpy.arange(0,1+1.0/n_bins,1.0/n_bins), alpha=1.0, color='r',linestyle='--')

        plt.savefig(calib_out_path + '/' + exp_name + '/' + plot_name, bbox_inches='tight')

        plt.cla()
        plt.close()

            

### Boxplots

In [7]:
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 20}
matplotlib.rc('font', **font)


label_map = {'logMSE':'log MSE','MSE':'MSE', 'ExpJacc':'MJICS'}

#ylims = {'MSE':(-0.05,.41),'ExpJacc':(0.47,1.0)} # for A & B
ylims = {'logMSE':(-7,2),'MSE':(-0.05,1.25),'ExpJacc':(0.25,1.05)} #for C & D

exp_pairs = [(a,b) for a in results for b in results if b.replace("'","")==a and a!=b]

# real data exps.
#exp_pairs = [("CompareMethods:real-A","CompareMethods:real-B'"), ("CompareMethods:real-C","CompareMethods:real-D'")]

for low_vif_exp, high_vif_exp in exp_pairs:
    low_name = low_vif_exp.replace('CompareMethods:sim-','') 
    high_name = high_vif_exp.replace('CompareMethods:sim-','').replace("'",'$_▵$') #▵

    out_path = './boxplots/' + low_vif_exp + '/'
    if not os.path.exists(out_path):
        os.makedirs(out_path)
        
    print('>>>',low_vif_exp, high_vif_exp)
    for metric in ['logMSE','ExpJacc']:
        
        data_dict = {'Model':[], 'Median VIF':[],metric:[]}
        print('>', metric)

        for model in results[low_vif_exp]['performance_scores_all']:
            for value in results[low_vif_exp]['performance_scores_all'][model][metric].values():
                    data_dict['Model'].append(model)
                    data_dict['Median VIF'].append(low_name)
                    data_dict[metric].append(value)
                    
        for model in results[high_vif_exp]['performance_scores_all']:
            for value in results[high_vif_exp]['performance_scores_all'][model][metric].values():
                    data_dict['Model'].append(model)
                    data_dict['Median VIF'].append(high_name)
                    data_dict[metric].append(value)
        boxplot_df = pd.DataFrame.from_dict(data_dict)
        plt.figure(figsize=(14, 10))

        
        cmap = matplotlib.cm.get_cmap('Set1')
        red = cmap.colors[0]
        blue = cmap.colors[1]
        legend_elements = [Patch(facecolor=blue, edgecolor='black', label=low_name),Patch(facecolor=red, edgecolor='black', label=high_name)]
        bp = sns.boxplot(x="Model", y=metric, hue='Median VIF',linewidth=1, width=.7, data=boxplot_df, palette=[blue,red],fliersize=0, hue_order=[low_name, high_name])
        sp = sns.stripplot(x="Model", y=metric, hue='Median VIF', palette=[blue,red],dodge=True, data=boxplot_df, alpha=0.15, hue_order=[low_name, high_name])
        sp.legend([],[], frameon=False)

        bp.set(xlabel=None)
        sp.set(xlabel=None)
        bp.set(ylabel=label_map[metric])
        
        plt.ylim(ylims[metric][0], ylims[metric][1])
        plt.legend(handles=legend_elements)

        plt.savefig(out_path +'/' + low_vif_exp + ':' + metric +'.pdf', bbox_inches='tight')
        print(out_path +'/' + low_vif_exp.replace(':','-') + '-' + metric +'.pdf')
        plt.cla()
        plt.close()

>>> CompareMethods:sim-D CompareMethods:sim-D'
> logMSE
./boxplots/CompareMethods:sim-D//CompareMethods-sim-D-logMSE.pdf
>>> CompareMethods:sim-C CompareMethods:sim-C'
> logMSE
./boxplots/CompareMethods:sim-C//CompareMethods-sim-C-logMSE.pdf
>>> CompareMethods:sim-A CompareMethods:sim-A'
> logMSE
./boxplots/CompareMethods:sim-A//CompareMethods-sim-A-logMSE.pdf
>>> CompareMethods:sim-B CompareMethods:sim-B'
> logMSE
./boxplots/CompareMethods:sim-B//CompareMethods-sim-B-logMSE.pdf


### Hyperparameter plots

In [17]:
import matplotlib.pyplot as plt

exp_pairs = [(a,b) for a in results for b in results if b.replace("'","")==a and a!=b]

params = []
hypp_df = pandas.DataFrame({'Collin':[],'EPV':[],'Hyperparameter':[],'Value':[],'Outcome':[],'PredSet':[]})

to_float = lambda x: float(x)
for low_exp, high_exp in exp_pairs:
    for exp_name in [low_exp, high_exp]:
        plot_exp_name = exp_name.replace('CompareMethods:sim-','').replace("'",'$_▵$') #▵
        predictor_set = plot_exp_name.replace('$_▵$','')
        collin = '▵' if '▵' in plot_exp_name else '▿'
        print(exp_name)
        outcome = 'xerostomia' if ('A' in plot_exp_name or 'B' in plot_exp_name) else 'dysphagia'
        no_preds = len(results[exp_name]['gt_models'][0].best_estimator_.coef_[0])
        

        for bs in results[exp_name]['bs_ixs']: # TODO remove 10
            
            num_events = sum(results[exp_name]['train_labs'][bs])[0]
            epv = round(num_events / no_preds,0)
            
            trained_models_path = results[exp_name]['bs_path'][bs] + '/trained_models.p'
            
            if os.path.exists(trained_models_path) and bs in results[exp_name]['gt_models']:
                with open(trained_models_path,'rb') as f:

                    for model_name, trained_models in pickle.load(f).items():
                        for model in trained_models:
                            
                            if model_name == 'Lasso':
                                hypp_df = hypp_df.append({'Outcome':outcome,'PredSet':predictor_set , 'EPV':epv, 'Collin':collin,'Hyperparameter': "CLasso",'Value':  to_float(model.best_estimator_.L1_C)},ignore_index=True)
                                hypp_df = hypp_df.append({'Outcome':outcome,'PredSet':predictor_set , 'EPV':epv, 'Collin':collin,'Hyperparameter': "LLasso",'Value': 1.0 / to_float(model.best_estimator_.L1_C)},ignore_index=True)
                            elif model_name == 'Ridge':
                                hypp_df = hypp_df.append({'Outcome':outcome,'PredSet':predictor_set, 'EPV':epv, 'Collin':collin, 'Hyperparameter': 'CRidge','Value': to_float(model.best_estimator_.L2_C)},ignore_index=True)
                                hypp_df = hypp_df.append({'Outcome':outcome,'PredSet':predictor_set, 'EPV':epv, 'Collin':collin, 'Hyperparameter': 'LRidge','Value': 1.0 / to_float(model.best_estimator_.L2_C)},ignore_index=True)
                            elif model_name == 'ElasticNet':
                                hypp_df = hypp_df.append({'Outcome':outcome,'PredSet':predictor_set, 'EPV':epv, 'Collin':collin,'Hyperparameter': 'CL1ENet','Value': to_float(model.best_estimator_.L1_C)},ignore_index=True)
                                hypp_df = hypp_df.append({'Outcome':outcome,'PredSet':predictor_set, 'EPV':epv, 'Collin':collin,'Hyperparameter': 'CL2ENet','Value':  to_float(model.best_estimator_.L2_C)},ignore_index=True)
                                hypp_df = hypp_df.append({'Outcome':outcome,'PredSet':predictor_set, 'EPV':epv, 'Collin':collin,'Hyperparameter': 'CENet','Value':  to_float(model.best_estimator_.L2_C+model.best_estimator_.L1_C)},ignore_index=True)
                                hypp_df = hypp_df.append({'Outcome':outcome,'PredSet':predictor_set, 'EPV':epv, 'Collin':collin,'Hyperparameter': 'LENet','Value':  to_float(1.0 / model.best_estimator_.L2_C+ 1.0 / model.best_estimator_.L1_C)},ignore_index=True)
                            elif model_name == 'PCLR':
                                hypp_df = hypp_df.append({'Outcome':outcome,'PredSet':predictor_set, 'EPV':epv, 'Collin':collin,'Hyperparameter': 'NC_PCA','Value': to_float(model.best_estimator_.n_components)},ignore_index=True)
                            elif model_name == 'LAELR':
                                hypp_df = hypp_df.append({'Outcome':outcome,'PredSet':predictor_set, 'EPV':epv, 'Collin':collin,'Hyperparameter': 'NC_LAE','Value': to_float(model.best_estimator_.LAE_h)},ignore_index=True)
                                hypp_df = hypp_df.append({'Outcome':outcome,'PredSet':predictor_set, 'EPV':epv, 'Collin':collin,'Hyperparameter': 'LAE_C','Value': to_float(model.best_estimator_.LAE_C)},ignore_index=True)
                                hypp_df = hypp_df.append({'Outcome':outcome,'PredSet':predictor_set, 'EPV':epv, 'Collin':collin,'Hyperparameter': 'LAE_L','Value': 1.0 / to_float(model.best_estimator_.LAE_C)},ignore_index=True)
                            elif model_name == 'Dropout':
                                hypp_df = hypp_df.append({'Outcome':outcome,'PredSet':predictor_set, 'EPV':epv, 'Collin':collin,'Hyperparameter': 'DR','Value': to_float(model.best_estimator_.dropout_ratio)},ignore_index=True)


                               
print(hypp_df)

CompareMethods:sim-D
CompareMethods:sim-D'
CompareMethods:sim-C
CompareMethods:sim-C'
CompareMethods:sim-A
CompareMethods:sim-A'
CompareMethods:sim-B
CompareMethods:sim-B'
      Collin  EPV Hyperparameter        Value     Outcome PredSet
0          ▿  2.0         CLasso     2.121903   dysphagia       D
1          ▿  2.0         LLasso     0.471275   dysphagia       D
2          ▿  2.0         CRidge     0.444094   dysphagia       D
3          ▿  2.0         LRidge     2.251776   dysphagia       D
4          ▿  2.0        CL1ENet   100.000000   dysphagia       D
...      ...  ...            ...          ...         ...     ...
10395      ▵  8.0         NC_PCA     5.000000  xerostomia       B
10396      ▵  8.0         NC_LAE     4.000000  xerostomia       B
10397      ▵  8.0          LAE_C     0.001000  xerostomia       B
10398      ▵  8.0          LAE_L  1000.000000  xerostomia       B
10399      ▵  8.0             DR     0.500000  xerostomia       B

[10400 rows x 6 columns]


In [18]:
import sys,os
sys.path.append(os.pathsep + '/usr/bin')

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
cmap = matplotlib.cm.get_cmap('Set1')
red = cmap.colors[0]
blue = cmap.colors[1]
from matplotlib import rc

matplotlib.rcParams['text.usetex'] = False
font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 20}
matplotlib.rc('font', **font)
out_path = './hypperparamplots/' 
if not os.path.exists(out_path):
        os.makedirs(out_path)
for outcome in ['xerostomia','dysphagia']:
    print(outcome)

    label_map = {'DR':'$\delta$','CRidge':'$c_{\ell_2}$','CLasso':'$c_{\ell_1}$','NC_PCA':'$d_{PCA}$','NC_LAE':'$d_{LAE}$','CENet':'$c_{ENet}$','LRidge':'$\lambda_{\ell_2}$','LLasso':'$\lambda_{\ell_1}$','LENet':'$\lambda_{ENet}$'}
    #label_map = {'LRidge':'$\lambda_{\ell_2}$','LLasso':'$\lambda_{\ell_1}$','LENet':'$\lambda_{ENet}$'}
    
    groups = [['LRidge',"LLasso","LENet"],['CRidge',"CLasso","CENet"],['DR'],['NC_LAE','NC_PCA']]
    sel_outcomes = [outcome]
    
    p = {'▵':red, '▿':blue}
    for selection in groups:
        f, ax = plt.subplots(figsize=(9, 9))
        lp = sns.lineplot(legend='brief',
        data=hypp_df[hypp_df['Hyperparameter'].isin(selection) &  hypp_df['Outcome'].isin(sel_outcomes)], ax=ax, x="PredSet", y="Value", style="Hyperparameter", hue="Collin", hue_order=['▵', '▿'], err_style='bars',err_kws={'capsize':5,'elinewidth':1}, ci=95, palette=[red, blue])                

        lp.set(xlabel=None,ylabel=None)
        leg = ax.legend()
        new_labels=[label_map[l] if l in label_map else l for l in selection]
        old_leg_handles = leg.legendHandles[-len(selection):]
        plt.legend(old_leg_handles, new_labels, title="")
        
        plt.savefig(out_path +'/' + outcome + ':' + '-'.join(selection) +'.pdf', bbox_inches='tight')
        print(out_path +'/' + outcome + ':' + '-'.join(selection) +'.pdf')
        plt.cla()
        plt.close()
        


xerostomia
./hypperparamplots//xerostomia:LRidge-LLasso-LENet.pdf
./hypperparamplots//xerostomia:CRidge-CLasso-CENet.pdf
./hypperparamplots//xerostomia:DR.pdf
./hypperparamplots//xerostomia:NC_LAE-NC_PCA.pdf
dysphagia
./hypperparamplots//dysphagia:LRidge-LLasso-LENet.pdf
./hypperparamplots//dysphagia:CRidge-CLasso-CENet.pdf
./hypperparamplots//dysphagia:DR.pdf
./hypperparamplots//dysphagia:NC_LAE-NC_PCA.pdf


#### Coefficient evaluation tables (LaTeX)

In [ ]:

def is_oar_coef(coef_name):
    for oar_substring in oar_coeff_substrings:
        if oar_substring in coef_name:
            return True
    return False

for exp_name in results:
    print(exp_name)    
    if 'umcg-cv' in exp_name: # Only include external evaluation settings for coefficient evaluation.
        print('...ignored')
        continue
        
    # 1. calculate summarizing coefficient statistics
    coeff_metrics = {model:{metric:{} for metric in coeff_metrics_to_include} for model in models_to_include}
    for bs in results[exp_name]['bs_ixs']:
        for model in coeff_metrics:
            first_coef_name = list(results[exp_name]['models'][model].keys())[0]
            if bs in results[exp_name]['models'][model][first_coef_name]:
                oar_coef_values = [results[exp_name]['models'][model][coef_name][bs] for coef_name in results[exp_name]['models'][model] if is_oar_coef(coef_name)]
                neg_oar_coef_values = [v for v in oar_coef_values if v < -0.01]
                pos_oar_coef_values = [v for v in oar_coef_values if v > 0.01]
                
                if 'sum_neg_oar' in coeff_metrics_to_include:
                    coeff_metrics[model]['sum_neg_oar'][bs] = sum(neg_oar_coef_values)
                if 'sum_pos_oar' in coeff_metrics_to_include:
                    coeff_metrics[model]['sum_pos_oar'][bs] = sum(pos_oar_coef_values)
                if 'perc_oar_neg' in coeff_metrics_to_include:
                    coeff_metrics[model]['perc_oar_neg'][bs] = len(neg_oar_coef_values) / len(oar_coef_values)
                if 'perc_oar_pos' in coeff_metrics_to_include:
                    coeff_metrics[model]['perc_oar_pos'][bs] = len(pos_oar_coef_values) / len(oar_coef_values)
                    
    # 2. Determine the reported scores, and confidence intervals
    main_scores, CI_lower, CI_upper = {model:{} for model in models_to_include}, {model:{} for model in models_to_include}, {model:{} for model in models_to_include}
    for model in coeff_metrics:
        for metric in coeff_metrics_to_include:
            sorted_scores = sorted(coeff_metrics[model][metric].values())
            main_scores[model][metric] = numpy.mean(list(coeff_metrics[model][metric].values()))
            
            CI_lower_ix = int(CI_dev*len(coeff_metrics[model][metric]))
            CI_upper_ix = int(math.ceil((1.0- CI_dev) * (len(coeff_metrics[model][metric])-1)))
            CI_lower[model][metric]=sorted_scores[CI_lower_ix]
            CI_upper[model][metric]=sorted_scores[CI_upper_ix]
        for coef_name in results[exp_name]['models'][model]:
            sorted_scores = sorted(results[exp_name]['models'][model][coef_name].values())

            main_scores[model][coef_name] = numpy.mean(list(results[exp_name]['models'][model][coef_name].values()))
            CI_lower_ix = int(CI_dev*len(results[exp_name]['models'][model][coef_name]))
            CI_upper_ix = int(math.ceil((1.0- CI_dev) * (len(results[exp_name]['models'][model][coef_name])-1)))
            CI_lower[model][coef_name]=sorted_scores[CI_lower_ix]
            CI_upper[model][coef_name]=sorted_scores[CI_upper_ix]            
 
    
    # save results to large  results dictionary
    results[exp_name]['coeff_scores'] = {'main':main_scores, 'ci_lower': CI_lower, 'ci_upper':CI_upper}
    results[exp_name]['coeff_scores_all'] = coeff_metrics


#### Write score tables (LaTeX)

In [ ]:

for exp_name in results:
    
    # Setup output dir
    out_file = top_level_out_dir + '/score_tables_tex/' + exp_name + '.tex'
    if not os.path.exists(os.path.dirname(out_file)):
        os.makedirs(os.path.dirname(out_file))
        
    # Write the results in a .tex table (only rows)
    score_formatting = lambda x: "{0:.2f}".format(round(x, 2))
    column_header_formatting = lambda x: x.replace('_mean','').replace('_','.')
    ci_formatting = lambda ci: "$^{(" + "{0:.2f}".format(round(ci[0], 2)) + ", " + "{0:.2f}".format(round(ci[1], 2)) + ")}$"

    with open(out_file, 'w') as f:
        all_metrics_to_include = metrics_to_include #+coeff_metrics_to_include    
        lines = "Model" + '&' + '&'.join([column_header_formatting(metric) for metric in all_metrics_to_include]) + '\\\\\n'
        for model in models_to_include:
            perf_score_content = [score_formatting(results[exp_name]['performance_scores']['main'][model][metric]) + ci_formatting((results[exp_name]['performance_scores']['ci_lower'][model][metric],results[exp_name]['performance_scores']['ci_upper'][model][metric])) for metric in metrics_to_include]
            #coeff_score_content = [score_formatting(results[exp_name]['coeff_scores']['main'][model][metric]) + ci_formatting((results[exp_name]['coeff_scores']['ci_lower'][model][metric],results[exp_name]['coeff_scores']['ci_upper'][model][metric])) for metric in coeff_metrics_to_include]
            all_score_content = perf_score_content #+ coeff_score_content
            lines += model + '&' + '&'.join(all_score_content) + '\\\\\n'
        f.write(lines)
    print('written',out_file)       




#### Coefficient tables (LaTeX)

In [ ]:
for exp_name in results:
    print(exp_name)
        
    # Setup output dir
    out_file = top_level_out_dir + '/coeff_tables/' + exp_name + '.tex'
    if not os.path.exists(os.path.dirname(out_file)):
        os.makedirs(os.path.dirname(out_file))
        
    # Write summarizing statistics and actual coefficients to tables
    column_header_formatting = lambda x: '\multicolumn{2}{l}{' +x.replace('_mean','') +'}'
    ci_formatting = lambda ci: "&$^{(" + "{0:.2f}".format(round(ci[0], 2)) + ", " + "{0:.2f}".format(round(ci[1], 2)) + ")}$"
    score_formatting = lambda x: "{0:.2f}".format(round(x, 2))
    row_formatting = lambda x: x.replace('X_','').replace('_mean','').replace('_','.').replace('Dmean','Dm').replace('UMCGshortv2','').replace('Submandibular','Subm')
    
    coef_names = results[exp_name]['models'][models_to_include[0]]
    with open(out_file, 'w') as f:
        lines = "\\begin{tabular}{lRlRlRlRlRlRlRlRlRlRlRlRlRlRlRlRlR}\\toprule\n"
        lines += " " + '&' + '&'.join([column_header_formatting(model) for model in models_to_include]) + '\EndTableHeader' + '\\\\\n'
        lines += "\\midrule "
        for coef_metric in coeff_metrics_to_include:
            lines += row_formatting(coef_metric)+ '&' + '&'.join([score_formatting(results[exp_name]['coeff_scores']['main'][model][coef_metric]) + ci_formatting((results[exp_name]['coeff_scores']['ci_lower'][model][coef_metric],results[exp_name]['coeff_scores']['ci_upper'][model][coef_metric])) for model in models_to_include]) + '\\\\\n'
            
        lines += "\\midrule "
        for coef_name in coef_names: #results[exp_name]['models'].keys():
            lines += row_formatting(coef_name) + '&' + '&'.join([score_formatting(results[exp_name]['coeff_scores']['main'][model][coef_name]) + ci_formatting((results[exp_name]['coeff_scores']['ci_lower'][model][coef_name],results[exp_name]['coeff_scores']['ci_upper'][model][coef_name])) for model in models_to_include]) + '\\\\\n'
        lines += "\\bottomrule\n\\end{tabular}"
        f.write(lines)
    print('written',out_file)       

In [ ]:
with open(top_level_out_dir +'/results.p', 'wb') as f:
    pickle.dump(results, f)
    print('saved results to', top_level_out_dir +'/results.p','\nDone!')


 ### Combined calibration plots

In [ ]:
from lib.utils import plot_calib_curves


for exp_name in results:

    # Setup output dir
    plot_path = top_level_out_dir + '/combined_calib_plots/' + exp_name + '.pdf'
    if not os.path.exists(os.path.dirname(plot_path)):
        os.makedirs(os.path.dirname(plot_path))    
    
    probs_per_model = {model: results[exp_name]['preds'][model][0] for model in results[exp_name]['preds'] if model in models_to_include}
    true_labs = results[exp_name]['labs'][0]
    
    plot_calib_curves(probs_per_model, true_labs, plot_path, n_bins=10, dpi=800)
    print('written',plot_path)

print('Done.')
    

### Correlation plots


In [ ]:
from exps.compare_methods_citor import prep_citor_data
from lib.CITOR import CITOR, load_python_object_encrypted
from lib.experiment import encode_variables, select_train_test_patients
import matplotlib.pyplot as plt


pwd='PWD'

score_formatting = lambda x: "{0:.2f}".format(round(x, 2))

header_formatting = lambda x: x.replace('X_','').replace('_mean','').replace('_','.').replace('Dmean','Dm').replace('UMCGshortv2','').replace('Submandibular','Subm').replace('..','.')
column_header_formatting = lambda x: '\\rotatebox{270}{\\multicolumn{1}{l}{' + header_formatting(x) + '}}'
row_header_formatting = header_formatting

score_coloring = lambda x: '\\multicolumn{1}{E}{' + x + '}'

for exp_name in results:
    used_data = prep_citor_data(load_python_object_encrypted(results[exp_name]['yaml']['data_path'],pwd))
    df, x_names, y_name = encode_variables(used_data, results[exp_name]['yaml'])
    df_unlabeled_train_indices, df_train_indices, df_unlabeled_test_indices, df_test_indices = select_train_test_patients(df, results[exp_name]['yaml'], y_name, x_names, bs=0)
    
    
    print(exp_name, len(x_names),len(df_train_indices), len(df_test_indices))
    corr_matrix = df.iloc[df_train_indices][x_names].corr()
    
    flattened_corr_matrix = corr_matrix.values.flatten()
    num_pairs_r_squared_larger_than_70_percent = (sum([1.0 if abs(v) > 0.7 else 0 for v in flattened_corr_matrix]) - len(x_names)) / 2

    print('no pairs |r| > 0.7:', num_pairs_r_squared_larger_than_70_percent,'/',len(flattened_corr_matrix),'=', num_pairs_r_squared_larger_than_70_percent/len(flattened_corr_matrix) )
    plt.matshow(corr_matrix)
    plt.show()

    # Setup output dir
    out_file = top_level_out_dir + '/correlation_tables/' + exp_name + '.tex'
    if not os.path.exists(os.path.dirname(out_file)):
        os.makedirs(os.path.dirname(out_file))
    
    
    with open(out_file, 'w') as f:
        string = "\\begin{tabular}{l|c" + (len(x_names) * 'c')+ '} \n\EndTableHeader \\\\ \n'
        
        for i, i_name in enumerate(x_names): # rows
            if i == 0:
                continue
                
            string += header_formatting(i_name) + '&&' 
            for j,j_name in enumerate(x_names): # columns
                if i > j:
                    string += score_coloring(score_formatting(corr_matrix.values[i,j])) + '&'
                else:
                    string += '&'
            string += '\\\\ \n'
            

        string += '\\midrule&\\textcolor{white}{.}&'+'&'.join([column_header_formatting(x_name) for x_name in x_names[:-1]]) + '\n'
            
        string += "\end{tabular} \n"
                
             
        f.write(string)        
    
    print('written', out_file)
    
print('Done.')
            

### Making the baseline table with patient characteristics

In [ ]:
xer_indices_umcg, xer_indices_maas = [],[]
dys_indices_umcg, dys_indices_maas = [],[]
y_names = set()

for exp_name in results:
    print('>>>',exp_name)
    used_data = prep_citor_data(load_python_object_encrypted(results[exp_name]['yaml']['data_path'],pwd))
    df, x_names, y_name = encode_variables(used_data, results[exp_name]['yaml'])
    df_unlabeled_train_indices, df_train_indices, df_unlabeled_test_indices, df_test_indices = select_train_test_patients(df, results[exp_name]['yaml'], y_name, x_names, bs=0)
    
    
    print('No pts. in total dataset', len(used_data))
    print('No pts. with M6 outcomes present', len(df_train_indices) + len(df_test_indices))
    
    print('UMCG data: n=',len(df_train_indices),'\n')

    if 'xer' in exp_name:
        xer_indices_umcg, xer_indices_maas = df_train_indices, df_test_indices
    if 'dys' in exp_name:
        dys_indices_umcg, dys_indices_maas = df_train_indices, df_test_indices  
    
    y_names.add(y_name)
umcg_both = [i for i in xer_indices_umcg if i in dys_indices_umcg]    
maas_both = [i for i in dys_indices_maas if i in xer_indices_maas]    

print('UMCG with both outcomes', len(umcg_both))
print('UMCG with either outcome', len(set(xer_indices_umcg + dys_indices_umcg)))
print('MAAS with both outcomes', len(maas_both))

In [ ]:
# To print: AGE (mean, std), GESLACHT (male & female), LOCTUM_cat (quantities per category)

for hosp_df, hosp_name in [(used_data.iloc[umcg_both],'UMCG'),(used_data.iloc[maas_both],'MAAS')]:
    print('\n\n',5*'=',hosp_name, 5*'=')
    print('AGE\n', 'Median',hosp_df['AGE'].median(), '\n Std.',hosp_df['AGE'].std())
    
    for k in ['GESLACHT','LOCTUM_cat','MODALITY','TCAT','NCAT'] +list(y_names):
        print(k)
        vk = hosp_df[k].value_counts()
        pk = hosp_df[k].value_counts(normalize=True)
        print(pandas.concat([vk,pk], axis=1, keys=['counts', '%']).sort_index())






